In [7]:
%pip install -Uq langchain langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kor 0.12.0 requires openai<0.28,>=0.27, but you have openai 1.9.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
# The simplest compostion is just combining a prompt and model to create a chain that takes user input, adds it to a prompt ,passes it to a model and returns the raw model output

In [21]:
prompt = ChatPromptTemplate.from_template("tell me some interesting facts about {foo}")
model = ChatOpenAI()
chain = prompt|model

In [22]:
chain.invoke({'foo':"paris"})

AIMessage(content='1. Paris is known as the "City of Love" and is one of the most romantic cities in the world.\n2. The Eiffel Tower, one of the most iconic landmarks in Paris, was originally intended to be a temporary structure for the 1889 World\'s Fair.\n3. Paris is divided into 20 different districts, known as arrondissements, each having its own unique character and charm.\n4. The Louvre Museum in Paris is the largest art museum in the world and houses over 38,000 objects, including the famous painting, Mona Lisa.\n5. Paris is home to several world-renowned fashion houses, such as Chanel, Dior, and Louis Vuitton.\n6. The city\'s underground catacombs hold the remains of approximately six million people and can be explored through a small section open to the public.\n7. Paris has a vibrant café culture, with over 7,000 cafes throughout the city.\n8. The Notre-Dame Cathedral, known for its stunning Gothic architecture, took nearly 200 years to complete and is a popular tourist attra

In [23]:
#Attaching Stop Sequences
#stops at \n
chain = prompt | model.bind(stop='\n')
chain.invoke({'foo':"Barcelona"})

AIMessage(content='1. Barcelona is the capital city of Catalonia, an autonomous community in Spain.')

In [28]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

In [41]:
chain = prompt | model.bind(function_call={'name':'joke'},functions=functions, stop="\n")

In [42]:
chain.invoke({'foo':'Tom'},config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{', 'name': 'joke'}})

In [31]:
#We can also add in an output parser to easily transform the raw LLM/ChatModel output into a more workable format

In [49]:
from langchain_core.output_parsers import StrOutputParser

chain = (prompt 
| model.bind(function_call={'name':'joke'},functions=functions)
| StrOutputParser()
)


#string output parser does the job of the print function

In [50]:
chain.invoke({"foo":"Lemon"})

''

In [56]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


chain = (prompt 
| model.bind(function_call={'name':'joke'},functions=functions)
| JsonOutputFunctionsParser()
)

#invoke the chain

chain.invoke({"foo":"Lemon"})

{'setup': 'Why did the lemon go to the doctor?',
 'punchline': "Because it wasn't peeling well!"}

In [57]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [58]:
chain.invoke({"foo":"Lemon"})

'Why did the lemon go to the doctor?'